### Libraries

In [18]:
%%capture
%reset -f                        # clear all variables from the workspace
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
import importlib
importlib.reload(utils)
from psutil import virtual_memory    
import datetime
import numpy as np                 

'machine learning imports'
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [19]:
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.12.0


### GPU

In [20]:
# if gpu available print name, else use cpu
if tf.test.is_gpu_available():
    print('GPU:', tf.test.gpu_device_name())
else:
    print('CPU:', tf.config.list_physical_devices('CPU'))

CPU: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2023-10-30 21:19:46.424417: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-30 21:19:46.424558: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Load Data

In [21]:
# Define the augmentation method and the data directory
AUGMENTATION = 'RealTabFormer'
data_dir = os.path.abspath('../data')

# Load the train and test datasets
df_train, df_test = utils.load_dataset(data_directory=data_dir, 
                                       augmentation=AUGMENTATION, 
                                       ignore_columns=['mbtcp.unit_id', 
                                                       'mbtcp.trans_id'])   

Loading complete.
Train data: 1500000 rows, 46 columns. 
Test data: 381934 rows, 46 columns.


### Data Preparation

In [22]:
# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label', 'Attack_type'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [23]:
# Encode the training and test labels if needed
X_train_enc, X_test_enc = utils.encode_categorical(X_train, X_test)

Categorical features to be encoded:

http.request.method
http.referer
mqtt.protoname
http.request.version
mqtt.topic
mqtt.conack.flags
dns.qry.name.len

Encoding complete.
No of features before encoding: 44
No of features after encoding: 83


#### Label Encoding

In [24]:
y_train_enc, y_test_enc, le = utils.encode_labels(y_train, y_test)
y_train_bin = tf.keras.utils.to_categorical(y_train_enc)
# y_test_bin = tf.keras.utils.to_categorical(y_test_enc)

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


#### Standardization of Data

In [25]:
X_train_scaled, X_test_scaled = utils.scale_data(X_train_enc, X_test_enc, scaler_type='standard')

          mean      std       
Train:    -0.000    0.944     
Test:     -0.014    1.865     


### Model Training

In [26]:
# Define the model
model = Sequential()
model.add(Dense(256, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(le.classes_), activation='softmax')) 

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

# ReduceLROnPlateau callback
monitor = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",
                                               factor=0.3,
                                               mode="min",
                                               patience=10,
                                               verbose=1,
                                               min_lr=1e-8)

# Checkpoint callback                                                
checkpoint = ModelCheckpoint(f'../checkpoints/neural_net/best_model_{AUGMENTATION}.h5', 
                              monitor='loss', 
                              save_best_only=True)

In [27]:
# Shuffle training data
X_train, y_train_bin = shuffle(X_train_scaled, y_train_bin, random_state=42)

# Train the model
history = model.fit(X_train, 
                    y_train_bin, 
                    epochs=100, 
                    batch_size=512, 
                    callbacks=[monitor, checkpoint])

Epoch 1/100
2930/2930 [==============================] - 19s 6ms/step - loss: 0.5926 - accuracy: 0.7361 - lr: 0.0010
Epoch 2/100
2930/2930 [==============================] - 17s 6ms/step - loss: 0.4625 - accuracy: 0.7888 - lr: 0.0010
Epoch 3/100
2930/2930 [==============================] - 17s 6ms/step - loss: 0.4521 - accuracy: 0.7929 - lr: 0.0010
Epoch 4/100
2930/2930 [==============================] - 18s 6ms/step - loss: 0.4463 - accuracy: 0.7951 - lr: 0.0010
Epoch 5/100
2930/2930 [==============================] - 19s 7ms/step - loss: 0.4433 - accuracy: 0.7961 - lr: 0.0010
Epoch 6/100
2930/2930 [==============================] - 19s 7ms/step - loss: 0.4393 - accuracy: 0.7977 - lr: 0.0010
Epoch 7/100
2930/2930 [==============================] - 19s 6ms/step - loss: 0.4355 - accuracy: 0.7992 - lr: 0.0010
Epoch 8/100
2930/2930 [==============================] - 18s 6ms/step - loss: 0.4369 - accuracy: 0.7982 - lr: 0.0010
Epoch 9/100
2930/2930 [==============================] - 18s 6ms

### Model Evaluation

In [ ]:
# predict probabilities for test set and get the index of the highest probability
predictions = model.predict(X_test_scaled)

11936/11936 [==============================] - 9s 741us/step


In [ ]:
# get the index of the highest probability
predictions = tf.argmax(predictions, axis=1).numpy()

In [ ]:
# Calculate metrics 
accuracy = metrics.accuracy_score(y_test_enc, predictions)
precision_m = metrics.precision_score(y_test_enc, predictions, average='macro')
recall_m = metrics.recall_score(y_test_enc, predictions, average='macro')
f1_score_m = metrics.f1_score(y_test_enc, predictions, average='macro')
precision_w = metrics.precision_score(y_test_enc, predictions, average='weighted')
recall_w = metrics.recall_score(y_test_enc, predictions, average='weighted')
f1_score_w = metrics.f1_score(y_test_enc, predictions, average='weighted')

In [ ]:
# create dictionary for results
results = {
    "model": "Neural Net",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision_macro": precision_m,
    "recall_macro": recall_m,
    "f1_macro": f1_score_m,
    "precision_weighted": precision_w,
    "recall_weighted": recall_w,
    "f1_weighted": f1_score_w
    }

# print results
utils.print_results_table(results)

╒══════════════════════╤═════════╕
│ Metric               │ Value   │
╞══════════════════════╪═════════╡
│ Accuracy             │ 93.85%  │
├──────────────────────┼─────────┤
│ Precision (macro)    │ 77.48%  │
├──────────────────────┼─────────┤
│ Recall (macro)       │ 80.99%  │
├──────────────────────┼─────────┤
│ F1 (macro)           │ 75.99%  │
├──────────────────────┼─────────┤
│ Precision (weighted) │ 95.41%  │
├──────────────────────┼─────────┤
│ Recall (weighted)    │ 93.85%  │
├──────────────────────┼─────────┤
│ F1 (weighted)        │ 94.12%  │
╘══════════════════════╧═════════╛


#### Save Metrics Results 

In [ ]:
# save results to csv   
utils.save_results_to_csv([results], '../results/metrics/neural_net.csv')

#### Confusion Matrix

In [ ]:
conf_mat = metrics.confusion_matrix(y_test_enc, predictions)

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = list(le.classes_), 
                            columns = list(le.classes_))
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'


# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4453,0,0,0,1,169,0,0,0,83,76,0,0,0,0
DDoS_HTTP,0,5791,0,0,0,0,0,0,0,0,0,0,0,5,3832
DDoS_ICMP,0,0,13428,0,1,72,0,0,0,0,0,0,0,0,0
DDoS_TCP,0,0,0,5816,0,0,0,0,0,4193,0,0,0,0,0
DDoS_UDP,0,0,0,0,24601,0,0,0,0,0,0,0,0,0,0
Fingerprinting,0,0,0,0,0,123,0,0,0,23,0,0,0,0,0
MITM,0,0,0,0,0,0,76,0,0,0,0,0,0,0,0
Normal,10,0,0,0,0,1,0,272762,0,0,0,0,0,0,3
Password,0,0,0,0,0,0,0,0,6383,0,0,2822,903,0,0
